### WORK IN PROGRESS
Includes Data Loading Techniques for CDS Data Loading <br>
=> loads data<br>
=> but cerra data is not in right format ? <br>
=> need to go over this again + write down questions for irene<br>

In [ ]:
from IPython.display import display
import cdsapi
import xarray as xr
from xarray import backends
from urllib.request import urlopen
import io

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from cartopy import crs as ccrs, feature as cfeature

import config.data_params
#from config.data_params import CERRA_PARAMS, ERA5_PARAMS, DATASET_CERRA, DATASET_ERA5
from visualization.climate_data_visualizer import ClimateDataVisualizer
import warnings
warnings.filterwarnings('ignore')

import importlib

## Input Data
**Era5 Data** <br>
displays the low-resolution data for the model, which should later be downscaled so it fits the high-resolution data.<br>

**Cerra Data** <br>
displays the high-resolution data for the model, it's the goal data format. 

In [ ]:
def load_climate_data(params, dataset, save_to_disk=False, load_from_disk=False):
    """
    Load climate data from CDS or disk.

    Parameters:
    - params (dict): Dictionary of parameters for CDS retrieval.
    - dataset (str): Name of the dataset.
    - save_to_disk (bool): Flag indicating whether to save the data to disk.
    - load_from_disk (bool): Flag indicating whether to load the data from disk.

    Returns:
    - xr.Dataset: Loaded climate data.
    """
    if load_from_disk:
        try:
            return load_data_from_disk(dataset)
        except FileNotFoundError:
            print(f"File {dataset} not found on disk. Retrieving data from CDS.")
            return retrieve_from_cds(dataset, params, save_to_disk)
        
    else:
        return retrieve_from_cds(dataset, params, save_to_disk)

    
def retrieve_from_cds(dataset, params, save_to_disk = False):
    """
    Retrieve climate data from CDS.

    Parameters:
    - dataset (str): Name of the dataset.
    - params (dict): Dictionary of parameters for CDS retrieval.
    - save_to_disk (bool): Flag indicating whether to save the data to disk.

    Returns:
    - xr.Dataset: Loaded climate data.
    """
    cds = cdsapi.Client()
    data = cds.retrieve(dataset, params, ) #'download.grib')

    if save_to_disk:
        store_data_to_disk(dataset, data)

    with urlopen(data.location) as d:
        bytes_ = d.read()
        ds = xr.open_dataset(io.BytesIO(bytes_)).load()
        #ds = xr.open_dataset(d, engine='netcdf4')

        # TODO try old version of retrieving without bytes
    
    return ds


def load_data_from_disk(file_name, file_path="./data/"):
    """
    Load climate data from disk.

    Parameters:
    - file_path (str): Path to the directory containing the file. Default is "data".
    - file_name (str): Name of the file.

    Returns:
    - xr.Dataset: Loaded climate data.
    """
    file_path = file_path + file_name + ".nc"
    return xr.open_dataset(file_path, engine='netcdf4')


def store_data_to_disk(file_name, data, folder_path="data"):
    """
    Store data to disk in a specified folder.

    Parameters:
    - file_name (str): The name of the file to be saved.
    - data: The data to be stored.
    - folder_path (str, optional): The path to the folder where the file will be saved.
      Default is "data".
    """
    data.download(f"./{folder_path}/{file_name}.nc")


In [ ]:
def add_dimensions_to_cerra(cerra_data, file_name="", store_to_disk = False):
    """
    Add longitude and latitude dimensions to a CERRA xarray dataset.

    Parameters:
    - cerra_data (xr.Dataset): CERRA xarray dataset.

    Returns:
    - xr.Dataset: Modified dataset with added longitude and latitude dimensions.
    """

    # Assure that longitude and latitude already exist as coordinates
    if 'longitude' in cerra_data.coords and 'latitude' in cerra_data.coords:
        print("Longitude and latitude coordinates already exist.")

    else:
        raise ValueError("Longitude and latitude coordinates are required but not found in the input dataset.")

    # Adding longitude and latitude as dimensions
    cerra_data = cerra_data.assign_coords(longitude=cerra_data['longitude'], latitude=cerra_data['latitude'])


    if store_to_disk and file_name:
        store_data_to_disk(file_name, cerra_data)

    return cerra_data



In [ ]:
# needed so jupyter cached is cleared 
importlib.reload(config.data_params)

print(config.data_params.CERRA_PARAMS)
cerra_ds = load_climate_data(config.data_params.CERRA_PARAMS, config.data_params.DATASET_CERRA, True)

#cerra_data = add_dimensions_to_cerra(cerra_ds, DATASET_CERRA, True)

In [ ]:
# needed so jupyter cached is cleared 
importlib.reload(config.data_params)

print(config.data_params.ERA5_PARAMS)

# LOAD ERA% Data
era5_dataset = load_climate_data(config.data_params.ERA5_PARAMS, config.data_params.DATASET_ERA5)

#display(era5_dataset)

# 1 J - 12 M - 31 D => 13 m 20.6 s 

### Loading Data

In [ ]:
cerra_dataset = load_data_from_disk(config.data_params.DATASET_CERRA)
era5_dataset = load_data_from_disk(config.data_params.DATASET_ERA5)


### Visualizig Data

In [ ]:
visualizer = ClimateDataVisualizer()
visualizer.plot_era5_data(era5_dataset)
visualizer.plot_cerra_data(cerra_dataset)

In [ ]:
# find coordinate range
print("----CERRA----")
print ("LONGITUDE")
print("min: ", cerra_dataset.longitude.min().values)
print("max: ", cerra_dataset.longitude.max().values)

print ("LATITUDE")
print("min: ", cerra_dataset.latitude.min().values)
print("max: ", cerra_dataset.latitude.max().values)

print("----ERA5----")
print ("LONGITUDE")
print("min: ", era5_dataset.longitude.min().values)
print("max: ", era5_dataset.longitude.max().values)

print ("LATITUDE")
print("min: ", era5_dataset.latitude.min().values)
print("max: ", era5_dataset.latitude.max().values)

### Modifying CERRA Data

##### Umbennen der Dimension in longitude & latitude 

In [ ]:
print(len(cerra_dataset.longitude))
print(len(cerra_dataset.latitude))
print(cerra_dataset.dims) # it seems like that 'y': 1069, 'x': 1069 are longitudue & latitude :D 
#display(cerra_dataset)

cerra_ds_old_version = cerra_dataset

# Benenne die 'x'-Dimension in 'longitude' um
cerra_dataset_renamed = cerra_dataset.rename({'x': 'longitude'})

# Benenne die 'y'-Dimension in 'latitude' um
cerra_dataset_renamed = cerra_dataset_renamed.rename({'y': 'latitude'})

In [ ]:
ds_with_index = cerra_dataset_renamed.set_index(longitude='longitude', latitude='latitude')

In [ ]:

old_cerra = load_data_from_disk("t2m_cerra_202001_005deg")
display(old_cerra)


In [ ]:
desired_time = '2020-01-01T12:00:00'  # Replace with your specific time
desired_latitude = 40  # Replace with your specific latitude
desired_longitude = 15 # Replace with your specific longitude

# Use .sel() to extract the data for the specified time, latitude, and longitude
old_desired = old_cerra['t2m'].sel(time=desired_time, latitude=desired_latitude, longitude=desired_longitude, method='nearest')
print("value of old dat set - t2m:       ", old_desired.values)
print("")

print(old_cerra.latitude)
print("")
print("")
print(cerra_dataset.latitude)
print("")
print("")



# Print the extracted data
desired_data_d = cerra_dataset['t2m'].sel(time=desired_time)
print("")

#print(desired_data_d.latitude.values[0].size)
print(desired_data_d.latitude.values[0])
print(desired_data_d.latitude.values[1])
print(desired_data_d.latitude.values[1068])
#print(desired_data_d.latitude.shape)

#print(desired_data_d.longitude.x)
print("")
#print(desired_data_d.longitude.y)
print("")
#print(desired_data_d.x)

print(desired_data_d)
#temperature_at_location = cerra_dataset.sel(latitude=desired_latitude, longitude=desired_longitude, method='nearest')
#print(temperature_at_location)

print(era5_dataset)


In [ ]:
ds = cerra_dataset

# Assuming your Dataset is named 'ds'
desired_latitude = 40
desired_longitude = 15

# Extract latitude and longitude values
latitude_values = ds.latitude.values
longitude_values = ds.longitude.values

# Find the index of the nearest latitude and longitude values
lat_idx = np.abs(latitude_values - desired_latitude).argmin()
lon_idx = np.abs(longitude_values - desired_longitude).argmin()

print(latitude_values - desired_latitude)

# Select temperature at the desired location
temperature_at_location = ds.t2m[:, lat_idx, lon_idx]

# Print the result
print(temperature_at_location)


In [ ]:
ds = cerra_dataset.squeeze(dim='latitude', drop=True)
ds = ds.squeeze(dim='longitude', drop=True)
display(ds)

In [ ]:
# Choose a specific latitude value for which you want to keep the longitude coordinate
selected_latitude = 20.37505942  # Replace with the actual latitude value you want

# Use .sel() to select the longitude coordinate for the chosen latitude
ds_subset = cerra_dataset.isel(latitude=selected_latitude)

# Now, ds_subset should have 'longitude' as a one-dimensional coordinate
longitude_values = ds_subset['longitude'].values


#### Crop Coordinates

In [ ]:

# Füge einen neuen Index für 'longitude' und 'latitude' hinzu
ds_with_index = cerra_dataset.set_index(longitude='longitude', latitude='latitude')


# Old Values
lon_min_cerra_old, lon_max_cerra_old= cerra_dataset['longitude'].min().values, cerra_dataset['longitude'].max().values
lat_min_cerra_old, lat_max_cerra_old = cerra_dataset['latitude'].min().values, cerra_dataset['latitude'].max().values

# Extrahiere die gewünschten Koordinatenbereiche von ERA5
lon_min_era5, lon_max_era5 = era5_dataset['longitude'].min().values, era5_dataset['longitude'].max().values
lat_min_era5, lat_max_era5 = era5_dataset['latitude'].min().values, era5_dataset['latitude'].max().values


# Verkleinere den Koordinatenbereich von CERRA
ds_cerra_subset = cerra_dataset.sel(
    longitude=slice(lon_min_era5, lon_max_era5),
    latitude=slice(lat_min_era5, lat_max_era5)
)

# new values
lon_min_cerra_new, lon_max_cerra_new = ds_cerra_subset['longitude'].min().values, ds_cerra_subset['longitude'].max().values
lat_min_cerra_new, lat_max_cerra_new = ds_cerra_subset['latitude'].min().values, ds_cerra_subset['latitude'].max().values


print('--- OLD ---')
print('longitude: ', lon_min_cerra_old, lon_max_cerra_old)
print('latitude: ', lat_min_cerra_old, lat_max_cerra_old )

print('--- NEW ---')
print('longitude: ', lon_min_cerra_new, lon_max_cerra_new)
print('latitude: ', lat_min_cerra_new, lat_max_cerra_new)



#### Add longitidue & latitude to dimension

In [ ]:
import xarray as xr
import cartopy.crs as ccrs

# Load the original dataset
data = xr.open_dataset('CERRA_original.nc')

# Define the EC projection
proj = ccrs.EquidistantCylindrical()

# Project the latitude and longitude coordinates
data['latitude'] = data['latitude'].assign_coords(projection=proj)
data['longitude'] = data['longitude'].assign_coords(projection=proj)

# Interpolate the data to the new projection
data = data.interp_like(data, method='linear', fill_value=np.nan)

# Save the modified dataset
data.to_netcdf('CERRA_modified_with_EC.nc')
